In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from __future__ import division
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
matplotlib.rcParams['savefig.dpi'] = 1.5 * matplotlib.rcParams['savefig.dpi']

import numpy as np
import math, sys, os, glob

import george
import george.kernels as kernels
import emcee
import corner

import cPickle as pickle

try:
    from IPython.core.display import clear_output
    have_ipython = True
except ImportError:
    have_ipython = False

Msol = 1.98855*10.0**30.0
yr = 365.25 * 86400.0

In [ ]:
## These are just some matplotlib formatting options

def figsize(scale):
    fig_width_pt = 469.755                          # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = fig_width*golden_mean              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

plt.rcParams.update(plt.rcParamsDefault)
params = {'backend': 'pdf',
        'axes.labelsize': 10,
        'lines.markersize': 4,
        'font.size': 10,
        'xtick.major.size':6,
        'xtick.minor.size':3,  
        'ytick.major.size':6,
        'ytick.minor.size':3, 
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'xtick.minor.width':0.5,
        'ytick.minor.width':0.5,
        'lines.markeredgewidth':1,
        'axes.linewidth':1.2,
        'legend.fontsize': 7,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'savefig.dpi':200,
        'path.simplify':True,
        'font.family': 'serif',
        'font.serif':'Times',
        'text.latex.preamble': [r'\usepackage{amsmath}'],
        'text.usetex':True,
        'axes.color_cycle': ['b', 'lime', 'r', 'purple', 'g', 'c', 'm', 'orange', 'darkblue', \
                                'darkcyan', 'y','orangered','chartreuse','brown','deeppink','lightgreen', 'k'],
        'figure.figsize': (3.39,2.1)}
plt.rcParams.update(params)

## Let's read in the population synthesis spectra. These spectra cover a range of stellar density values and initial binary eccentricities.

In [ ]:
filnames = glob.glob('./data/*.gz')

In [ ]:
aa = []
bb = []
for ii in range(len(filnames)):
    aa.append(filnames[ii].split('_')[-3].split('Rho')[1])
    bb.append(filnames[ii].split('_')[-2].split('Ecc')[1])

In [ ]:
aa = np.unique(np.array(aa))
bb = np.unique(np.array(bb))

aa = aa[np.argsort(aa.astype(float))]

In [ ]:
data = []
ct = 0
for stars in aa:
    for ecc in bb:
        data.append(np.loadtxt('./data/Spectra_Rho{0}_Ecc{1}_HundredYears.gz'.format(str(stars),str(ecc))))
        print ct, stars, ecc
        ct += 1

In [ ]:
data = np.array(data)

aa = aa.astype(float) / 100.0 # log10 of stellar density
bb = bb.astype(float) / 1000.0 # eccentricity

## First of all, let's just take a look at the spectra from the extreme corners of astrophysical environmental conditions.

## This is Fig. 1 of Taylor, Simon, Sampson (2016)

In [ ]:
inds = [0,13,168,181] 
labs = ['\\textbf{(a)}', '\\textbf{(b)}', '\\textbf{(c)}', '\\textbf{(d)}']

fig,ax = plt.subplots(nrows=2,ncols=2,sharex=True,sharey=True)

ctx=0
cty=0
for kk,ind in enumerate(inds):

    newfreqs = np.arange(1.0,90.0) / (30.0*365.25*86400.0)
    newwidth = newfreqs[1]-newfreqs[0]
    newstrain = np.zeros((len(newfreqs),100))

    finefreqs = data[ind,0,:]
    finestrain = data[ind,1:,:]
    finewidth = finefreqs[1] - finefreqs[0]
    
    for ii in range(len(newfreqs)):
        for jj in range(data.shape[1]-1):
            
            mask = np.logical_and(finefreqs >= (newfreqs[ii] - newwidth/2.), finefreqs <= (newfreqs[ii] + newwidth/2.))
            newstrain[ii,jj] = np.sum(finestrain.T[mask,jj]) * finewidth / newwidth 

    # compute mean and std from original binning
    mean = np.mean(np.log10(np.sqrt(data[ind,:,:])),axis=0)
    err = np.std(np.log10(np.sqrt(data[ind,1:,:])),axis=0)

    # compute mean and std from new binning
    newmean = np.mean(np.log10(np.sqrt(newstrain)),axis=1)
    newerr = np.std(np.log10(np.sqrt(newstrain)),axis=1)
    
    if ind == 0:
        ctx,cty = 0, 0
    elif ind == 13:
        ctx,cty = 0, 1
    elif ind == 168:
        ctx,cty = 1, 0
    elif ind == 181:
        ctx,cty = 1, 1

    ax[ctx,cty].plot(newfreqs, 10.0**newmean, color='r', alpha=0.5, label=labs[kk])
    ax[ctx,cty].plot(newfreqs, 6e-16 * (newfreqs * 365.25*86400.0)**(-2./3.), 
             color='b', ls='dashed', alpha=0.5)
    
    for ii in range(100):
        ax[ctx,cty].plot(data[ind,0,:], np.sqrt(data[ind,1+ii,:]), color='k', lw=1.2, alpha=0.01)
    
    ax[ctx,cty].set_xscale('log')
    ax[ctx,cty].set_yscale('log')
    ax[ctx,cty].set_xlim(1e-9,1.0/(365.25*86400.0))
    ax[ctx,cty].set_ylim(1e-16,2e-14)
    leg = ax[ctx,cty].legend(loc='upper right',frameon=False,handlelength=0, handletextpad=0)
    for item in leg.legendHandles:
        item.set_visible(False)
    
plt.subplots_adjust(wspace=0, hspace=0)
plt.text(2e-10,1.5e-17,'GW Frequency [Hz]')
plt.text(8e-12,1e-12,'Characteristic strain, $h_c(f)$',rotation=90)
plt.show()

## We have the characteristic strain spectra starting at 1/(100-yrs). Let's re-bin the spectra for a 30-yr baseline instead of 100-yrs. 

## Also, let's just look at the characteristic strain at one frequency, f_gw = 1/(30-yrs)

In [ ]:
freq_ind = 0

newfreqs = np.arange(1.0,100.0) / (30.0*365.25*86400.0)
newwidth = newfreqs[1]-newfreqs[0]
newstrain = np.zeros((data.shape[0],data.shape[1]-1))

finefreqs = data[0,0,:] # same for all parameter inputs
finestrain = data[:,1:,:]
finewidth = finefreqs[1] - finefreqs[0]

for ii in range(data.shape[0]):
    for jj in range(data.shape[1]-1):
        
        mask = np.logical_and(finefreqs >= (newfreqs[freq_ind] - newwidth/2.), 
                              finefreqs <= (newfreqs[freq_ind] + newwidth/2.))
        
        newstrain[ii,jj] = np.sum(finestrain[ii,jj,mask]) * finewidth / newwidth 

## Compute mean strain spectrum and std (from Poissonian realization variance) from new binning. This becomes our data.

In [ ]:
newmean = np.mean(np.log10(np.sqrt(newstrain)/1e-15),axis=1)
newerr = np.std(np.log10(np.sqrt(newstrain)/1e-15),axis=1)

In [ ]:
# Sort data in 2-d space of density and eccentricity into one long vector

xobs = np.zeros((len(aa)*len(bb),2))
    
A, B = np.meshgrid(aa,bb)

xobs[:,0] = A.flatten(order='F') # iterates over all eccentricities then all stellar densities
xobs[:,1] = B.flatten(order='F')

yobs = newmean
yerr = newerr

## Now we have the data. So let's set up the Gaussian Process (GP), using the George GP regression library.

In [ ]:
# We use a Squared-Exponential kernel

k = 1.0 * kernels.ExpSquaredKernel([0.5,0.5],ndim=2)

In [ ]:
# Instantiate the GP

gp = george.GP(k)

In [ ]:
# Pre-compute the factorization of the matrix.

gp.compute(xobs, yerr)

In [ ]:
# Compute the log likelihood of the data, given the GP model.

print(gp.lnlikelihood(yobs))

## OK. Let's try predicting new values for the strain spectrum between the training points, i.e. at new values of stellar-density and eccentricity.

In [ ]:
# Define prediction points

t = np.zeros((10000,2))
    
xx = np.linspace(1.0,4.0,100) # log10(\rho / Msol.pc^{-3}) in U[1.0,4.0] 
yy = np.linspace(0.0,0.95,100) # e_0 in U[1.0,4.0] 

X, Y = np.meshgrid(xx,yy)

t[:,0] = X.flatten(order='F')
t[:,1] = Y.flatten(order='F')

## Predict the characteristic strain across astrophysical parameter space, and give a prediction uncertainity.

In [ ]:
mu, cov = gp.predict(yobs, t)
std = np.sqrt(np.diag(cov))

## The above is fine, but really we should train our GP kernel parameters on the simulation data. So, let's optimize the kernel parameters.

In [ ]:
# Define a GP class containing the kernel parameter priors and a log-likelihood

class gaussproc(object):
    
    def __init__(self, x, y, yerr=None):
        
        self.x = x
        self.y = y
        self.yerr = yerr
        
        self.pmax = np.array([20.0, 20.0, 20.0])
        self.pmin = np.array([-20.0, -20.0, -20.0])
        self.emcee_flatchain = None
        self.emcee_flatlnprob = None
        self.emcee_kernel_map = None
    
    def lnprior(self, p):
    
        logp = 0.
    
        if np.all(p <= self.pmax) and np.all(p >= self.pmin):
            logp = np.sum(np.log(1/(self.pmax-self.pmin)))
        else:
            logp = -np.inf

        return logp

    def lnlike(self, p):

        # Update the kernel and compute the lnlikelihood.
        a, tau = np.exp(p[0]), np.exp(p[1:3])
        
        lnlike = 0.0
        try:
            gp = george.GP(a * kernels.ExpSquaredKernel(tau,ndim=2))
            #gp = george.GP(a * kernels.Matern32Kernel(tau))
            gp.compute(self.x , self.yerr)
            
            lnlike = gp.lnlikelihood(self.y, quiet=True)
        except np.linalg.LinAlgError:
            lnlike = -np.inf
        
        return lnlike
    
    def lnprob(self, p):
        
        return self.lnprior(p) + self.lnlike(p)

In [ ]:
# Instanciate a GP again 

gp_george = gaussproc(xobs,yobs,yerr)
    
k = 1.0 * kernels.ExpSquaredKernel([2.0,2.0],ndim=2)
num_kpars = len(k) 

In [ ]:
# Use emcee (http://dan.iel.fm/emcee/current/) to sample the 
# posterior distribution of the kernel parameters to find MAP value

nwalkers, ndim = 36, num_kpars
sampler = emcee.EnsembleSampler(nwalkers, ndim, gp_george.lnprob)

# Initialize the walkers.
p0 = [np.log([1.,1.,1.]) + 1e-4 * np.random.randn(ndim)
      for i in range(nwalkers)]

print "Running burn-in"
p0, lnp, _ = sampler.run_mcmc(p0, 500)
sampler.reset()

print "Running second burn-in"
p = p0[np.argmax(lnp)]
p0 = [p + 1e-8 * np.random.randn(ndim) for i in xrange(nwalkers)]
p0, _, _ = sampler.run_mcmc(p0, 500)
sampler.reset()

print "Running production"
p0, _, _ = sampler.run_mcmc(p0, 1000)

In [ ]:
# Let's take a look at the posterior distribution of these kernel parameters.
# The second and third parameters are the natural-log of the kernel length-scales
# in the stellar-density and eccentricity parameters, respectively.

fig = corner.corner(sampler.flatchain,bins=50)
plt.show()

In [ ]:
# Let's grab the MAP values

mapparams = np.exp(sampler.flatchain[np.argmax(sampler.flatlnprobability)])

In [ ]:
# Set up a GP kernel with the MAP values found from the sampling.

k = mapparams[0] * kernels.ExpSquaredKernel(mapparams[1:],ndim=2)

# Instanciate the GP.
gp = george.GP(k)

# Pre-compute the factorization of the matrix.
gp.compute(xobs, yerr)

# Compute the log likelihood.
print(gp.lnlikelihood(yobs))

In [ ]:
# Predict the strain at new astrophysical parameter locations between the training.
# This time though, we use the fully trained GP.

mu, cov = gp.predict(yobs, t)
std = np.sqrt(np.diag(cov))

## Great, so let's now plot the grid of training points, the new predictions across parameter space from the trained GP, and the prediction uncertainties.

## This is Fig. 2 from Taylor, Simon, Sampson (2016). For full details, see the relevant figure caption in the paper.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2) 

im1 = axes[0].imshow(mu.reshape((100,100),order='F')-15.0,extent=(xx.min(),xx.max(),yy.max(),yy.min()),
               cmap=matplotlib.cm.cubehelix, aspect=3.5)#, vmin=0.0579768135068413, vmax=1.0)
im2 = axes[1].imshow(std.reshape((100,100),order='F'),extent=(xx.min(),xx.max(),yy.max(),yy.min()),
               cmap=matplotlib.cm.cubehelix, aspect=3.5)#, vmin=0.0579768135068413, vmax=1.0)

for ii in range(xobs.shape[0]):
    axes[0].scatter(x=xobs[ii,0],y=xobs[ii,1],
                    alpha=0.4,color='r',s=5,marker='.',clip_on=False)

for ax in axes.flat:
    ax.minorticks_on() 
    ax.set_xlim(xx.min(),xx.max())
    ax.set_ylim(yy.max(),yy.min())
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)

plt.setp(axes[1].get_yticklabels(), visible=False)

axes[0].set_ylabel(r'$e_0$')

fig.subplots_adjust(wspace=0.2,right=0.9)
cbar_ax = fig.add_axes([0.125, 0.85, 0.35, 0.04])
fig.colorbar(im1, cax=cbar_ax, orientation='horizontal')
cbar_ax.set_xticklabels(cbar_ax.get_xticklabels(),rotation=45,size=7)
cbar_ax.xaxis.set_ticks_position('top')

cbar_ax = fig.add_axes([0.55, 0.85, 0.35, 0.04])
fig.colorbar(im2, cax=cbar_ax, orientation='horizontal')
cbar_ax.set_xticklabels(cbar_ax.get_xticklabels(),rotation=45,size=7)
cbar_ax.xaxis.set_ticks_position('top')

plt.text(-0.5,-22.0,r'$\log_{10}(\rho / M_\odot\mathrm{pc}^{-3})$')
plt.text(-1.1,5.0,r'$\log_{10}[h_c(f=1\;\mathrm{nHz})]$',fontsize=8.5)
plt.text(0.06,5.0,r'$\Delta \log_{10}[h_c(f=1\;\mathrm{nHz})]$',fontsize=8.5)


from matplotlib.patches import Ellipse   
ellipse = Ellipse(xy=(2.5, 0.475), width=10.02123372/1., height=0.82797949/1., 
                  edgecolor='r', fc='None', lw=1.5, alpha=0.8)
axes[1].add_patch(ellipse)

plt.show()
#fig.savefig('gp4ptas_starsecc_trained.pdf',bbox_inches='tight',dpi=400,rasterized=True)

# We now have the full formalism in place. Let's repreat the above for all frequencies in the GW spectrum, not just the first.

In [ ]:
Tobs = 30.0 # years
newfreqs = np.arange(1.0,61.0) / (Tobs*365.25*86400.0)

newwidth = newfreqs[1]-newfreqs[0]
newstrain = np.zeros((data.shape[0],data.shape[1]-1))

finefreqs = data[0,0,:] # same for all parameter inputs
finestrain = data[:,1:,:]
finewidth = finefreqs[1] - finefreqs[0]

In [ ]:
# Packs the 2-d data into one long 1-d vector

xobs = np.zeros((len(aa)*len(bb),2))
A, B = np.meshgrid(aa,bb)
xobs[:,0] = A.flatten(order='F') # iterates over all eccentricities then all stellar densities
xobs[:,1] = B.flatten(order='F')

In [ ]:
## Re-bin the spectra from a baseline of 100-yrs to 30-yrs.
## This time for all frequencies, 1/(30-yrs) up to 60/(30-yrs).

## Note that we have divided through by 4.5e-31 which is the 
## squared-strain at 1/yr.

yobs = []
yerr = []
for freq_ind in range(len(newfreqs)):
    for ii in range(data.shape[0]):
        for jj in range(data.shape[1]-1):
            mask = np.logical_and(finefreqs >= (newfreqs[freq_ind] - newwidth/2.), 
                                  finefreqs <= (newfreqs[freq_ind] + newwidth/2.))
            newstrain[ii,jj] = np.sum(finestrain[ii,jj,mask]) * finewidth / newwidth 

    # compute mean and std from new binning
    newmean = np.log10(np.mean(newstrain/4.5e-31,axis=1)) 
    
    # cut out bad data, where some population realizations 
    # give zero strain at certain frequencies
    logstrain = np.log10(newstrain/4.5e-31)
    logstrain[logstrain==-np.inf] = 0.0
    
    newerr =  np.std(logstrain,axis=1) 
    
    yobs.append(newmean)
    yerr.append(newerr)

In [ ]:
# Instanciate a list of GP kernels and models [one for each frequency]

gp_george = []
k = []

for freq_ind in range(len(newfreqs)):
    
    gp_george.append(gaussproc(xobs,yobs[freq_ind],yerr[freq_ind]))
    k.append( 1.0 * kernels.ExpSquaredKernel([2.0,2.0],ndim=2) )
    num_kpars = len(k[freq_ind])

In [ ]:
# Sample the posterior distribution of the kernel parameters 
# to find MAP value for each frequency. 

# THIS WILL TAKE A WHILE... (~ 1 hr)

sampler = [0.0]*len(newfreqs)
for freq_ind in range(len(newfreqs)):
    
    # Set up the sampler.
    nwalkers, ndim = 36, num_kpars
    sampler[freq_ind] = emcee.EnsembleSampler(nwalkers, ndim, gp_george[freq_ind].lnprob)

    # Initialize the walkers.
    p0 = [np.log([1.,1.,1.]) + 1e-4 * np.random.randn(ndim)
          for i in range(nwalkers)]

    print freq_ind, "Running burn-in"
    p0, lnp, _ = sampler[freq_ind].run_mcmc(p0, 500)
    sampler[freq_ind].reset()

    print freq_ind, "Running second burn-in"
    p = p0[np.argmax(lnp)]
    p0 = [p + 1e-8 * np.random.randn(ndim) for i in xrange(nwalkers)]
    p0, _, _ = sampler[freq_ind].run_mcmc(p0, 500)
    sampler[freq_ind].reset()

    print freq_ind, "Running production", '\n'
    p0, _, _ = sampler[freq_ind].run_mcmc(p0, 1000)

In [ ]:
## Let's take a look at the posterior distribution of the 
# kernel parameters at a frequency [ind] of our choice.

ind = 59

fig = corner.corner(sampler[ind].flatchain,bins=50)
plt.show()

In [ ]:
## Populate the GP class with the details of the kernel 
## MAP values for each frequency.

for ii in range(len(newfreqs)):
    
    gp_george[ii].chain = None 
    gp_george[ii].lnprob = None 
    
    gp_george[ii].kernel_map = sampler[ii].flatchain[np.argmax(sampler[ii].flatlnprobability)] 
    print ii, gp_george[ii].kernel_map

# Store GP object for each frequency as a pickle. 

# This pickle object is now ready to be used in NX01 to constrain astrophysical parameters using real PTA data.

In [ ]:
pickle.dump(gp_george, open( "starsecc_gp_13x14nodes_30yr.pkl", "wb" ))

## Final plot. Provide a stellar-density and eccentricity, and let's look at the strain-spectrum.

In [ ]:
strain_predict = []
t = [np.array([3.35,0.65])] # log10(\rho) = 3.35, e_0 = 0.65

for ii in range(len(newfreqs)):
    
    # Let's grab the MAP values
    mapparams = np.exp(sampler[ii].flatchain[np.argmax(sampler[ii].flatlnprobability)])
    
    # Set up a GP kernel with the MAP values found from the sampling.
    k = mapparams[0] * kernels.ExpSquaredKernel(mapparams[1:],ndim=2)
    
    # Instanciate the GP.
    gp = george.GP(k)
    
    # Pre-compute the factorization of the matrix.
    gp.compute(xobs, yerr[ii])
    
    mu, cov = gp.predict(yobs[ii], t)
    std = np.sqrt(np.diag(cov))
    strain_predict.append(mu)

In [ ]:
plt.loglog(newfreqs[:20], np.sqrt(4.5e-31 * 10.0**np.array(strain_predict[:20])),
          alpha=0.5)
plt.xlabel(r'GW Frequency [Hz]')
plt.ylabel(r'Characteristic strain, $h_c(f)$')
plt.show()